# Weights, Biases and Layers implementation

With automatic backprop (differentiation with approximation)

In [1]:
import numpy as np # can replace with cupy
import random

In [2]:
# PREP FOR TESTING

# binary crossentropy
def CEloss(p_s0, p_s1, y_0, y_1, eps=1e-7):
  #print(p_s0, p_s1, y_0, y_1, eps)
  return (-1/np.log(2)) * (y_0*np.log(p_s0+eps) + y_1*np.log(p_s1+eps))


# to help turn gradients after function to gradients before function
def calc_grads_intinputs(fn, vals:list, ress:list, dx=1e-8, fn_out=None): # dx < eps # actually it's float inputs
  #This only works if the function takes in int inputs like fn(*args):
  # note that fn should return a numpy array so that it can be, like, subtracted
  if fn_out is None:
    fn_out = fn(*vals, *ress)
  grads = []
  for vidx in range(len(vals)):
    plus = fn(*vals[:vidx], vals[vidx]+dx, *vals[vidx+1:], *ress)
    minus = fn(*vals[:vidx], vals[vidx]-dx, *vals[vidx+1:], *ress)
    # (plus - minus)/(2*dx) would give d(fn)/d(x)
    grads.append((plus - minus)/(2*dx))
  return grads


# to help turn gradients after function to gradients before function
def calc_grads_nparr_input(fn, all_vals:np.array, vary, dx=1e-8, fn_out=None): # dx < eps
  #This only works if the function takes in int inputs like fn(*args):
  # note that fn should return a numpy array so that it can be, like, subtracted

  all_vals = all_vals.astype(np.float64) # increase precision for grad calculation

  if fn_out is None:
    fn_out = fn(all_vals)
  grads = []

  # turn vary into a list of the indices if it's a bool array
  if (len(vary)) > 0 and (isinstance(vary[0], bool)):
    inds = []
    for v in range(len(vary)):
      if vary[v]:
        inds.append(v)
    vary = inds

  for vidx in vary:
    add = np.array([int(i==vidx) for i in range(len(all_vals))])
    plus = fn(all_vals + add*dx)
    minus = fn(all_vals - add*dx)
    # (plus - minus)/(2*dx) would give d(fn)/d(x)
    grads.append((plus - minus)/(2*dx))
  return grads


# test it out
calc_grads_intinputs(CEloss, [1,0], [1,0])

[-1.4426949038686683, 0.0]

In [3]:
class Weight:
  def __init__(self, _w, _dtype=np.float32):
    self.dtype = _dtype
    self.w = np.array(_w, dtype=_dtype)
    self.last_x = np.array(0, dtype=_dtype)
    self.m = np.array(0, dtype=_dtype)
    self.v = np.array(0, dtype=_dtype)
    self.t = 0
    self.clear_grad()

  def mul(self, x):
    self.last_x = x
    return x * self.w

  def grad_from(self, next):
    self.grad += next.grad

  def finalize_grad(self):
    self.grad = (self.grad*self.last_x).reshape(())

  def update_w(self, lr):
    self.w -= self.grad * lr

  def adam_update_w(self, lr, beta1, beta2, t=None, m_prev=None, v_prev=None, eps=1e-7):
    # from https://medium.com/@weidagang/demystifying-the-adam-optimizer-in-machine-learning-4401d162cb9e

    if m_prev is None: m_prev = self.m
    if v_prev is None: v_prev = self.v
    if t is None: t = self.t

    # Update biased first moment estimate.
    # m is the exponentially moving average of the gradients.
    # beta1 is the decay rate for the first moment.
    m = beta1 * m_prev + (1 - beta1) * self.grad

    # Update biased second raw moment estimate.
    # v is the exponentially moving average of the squared gradients.
    # beta2 is the decay rate for the second moment.
    v = beta2 * v_prev + (1 - beta2) * self.grad**2

    # Compute bias-corrected first moment estimate.
    # This corrects the bias in the first moment caused by initialization at origin.
    m_hat = m / (1 - beta1**(t + 1))

    # Compute bias-corrected second raw moment estimate.
    # This corrects the bias in the second moment caused by initialization at origin.
    v_hat = v / (1 - beta2**(t + 1))

    # Update parameters.
    # Parameters are adjusted based on the learning rate, corrected first moment,
    # and the square root of the corrected second moment.
    # epsilon is a small number to avoid division by zero.
    self.w -= lr * m_hat / (np.sqrt(v_hat) + eps)

    # Save the first and second moment estimates.
    self.m = m
    self.v = v
    self.t += 1

  def clear_grad(self):
    self.grad = np.array(0, dtype=self.dtype) # grad is dL/d[]

  def __str__(self):
    return "Weight of "+str(self.w)+", dtype: "+str(self.dtype)



class Bias:
  def __init__(self, _b, _dtype=np.float32):
    self.dtype = _dtype
    self.b = np.array(_b, dtype=_dtype)
    self.grad = np.array(0, dtype=_dtype)
    self.m = np.array(0, dtype=_dtype)
    self.v = np.array(0, dtype=_dtype)
    self.t = 0
    self.clear_grad()

  def add(self, x):
    return self.b + x

  def grad_from(self, next):
    self.grad += next.grad

  def update_b(self, lr):
    self.b -= self.grad * lr # NOTE i didnt times lr last time i ran this so...

  def adam_update_b(self, lr, beta1, beta2, t=None, m_prev=None, v_prev=None, eps=1e-7):
    # from https://medium.com/@weidagang/demystifying-the-adam-optimizer-in-machine-learning-4401d162cb9e

    if m_prev is None: m_prev = self.m
    if v_prev is None: v_prev = self.v
    if t is None: t = self.t

    # Update biased first moment estimate.
    # m is the exponentially moving average of the gradients.
    # beta1 is the decay rate for the first moment.
    m = beta1 * m_prev + (1 - beta1) * self.grad

    # Update biased second raw moment estimate.
    # v is the exponentially moving average of the squared gradients.
    # beta2 is the decay rate for the second moment.
    v = beta2 * v_prev + (1 - beta2) * self.grad**2

    # Compute bias-corrected first moment estimate.
    # This corrects the bias in the first moment caused by initialization at origin.
    m_hat = m / (1 - beta1**(t + 1))

    # Compute bias-corrected second raw moment estimate.
    # This corrects the bias in the second moment caused by initialization at origin.
    v_hat = v / (1 - beta2**(t + 1))

    # Update parameters.
    # Parameters are adjusted based on the learning rate, corrected first moment,
    # and the square root of the corrected second moment.
    # epsilon is a small number to avoid division by zero.
    self.b -= lr * m_hat / (np.sqrt(v_hat) + eps)

    # Save the first and second moment estimates.
    self.m = m
    self.v = v
    self.t += 1

  def clear_grad(self):
    self.grad = np.array(0, dtype=self.dtype) # grad is dL/d[]

  def __str__(self):
    return "Bias of "+str(self.b)+", dtype: "+str(self.dtype)


class LayerFunc: # for functions like activation functions or softmax/pooling/similar
  # this function's format needs to be like sigmoid(x1, x2, ..., x10), so like use *args please
  EPS = 1e-7

  def __init__ (self, n_in, n_out, fun, _dtype=np.float32):
    self.n_in = n_in
    self.n_out = n_out
    self.fun = fun
    self.dtype = _dtype

  # for this to work, if n_out>1, self.fun must return an np.array
  def get_in_grads(self, in_vals, out_grads:np.array, vary:list=None): # turn gradients ater func to gradients before func
    # if vary is None, then vary all
    if vary is None:
      vary = list(range(len(in_vals)))

    rel_grads = calc_grads_nparr_input(self.fun, in_vals, vary)
    # this grads is d(out)/dx
    # so we should take grads @ final_grads for each x

    # just in case
    out_grads = np.array(out_grads)

    #print("REL:", rel_grads, "OUT:", out_grads)

    if self.n_out == 1:
      return rel_grads @ out_grads.T

    #print(rel_grads[0].shape, out_grads.shape)

    grads = [rel_grads[i] @ out_grads.T for i in range(len(rel_grads))]
    return grads

  def __call__(self, in_vals):
    #print("CALLING LAYERFUNC ON", in_vals)
    return self.fun(in_vals)



class EmptyLayerFunc(LayerFunc):
  def __init__(self, num, _dtype=np.float32):
    self.n_in = num
    self.n_out = num
    self.fun = lambda x:x
    self.dtype = _dtype

  def get_in_grads(self, in_vals, in_consts, out_grads:np.array):
    return out_grads # sincei t's empty so the rest can be ignored

  def __call__(self, x):
    return x



class Layer:
  def __init__(self, ins, outs, weight_gen, bias_gen, activation:LayerFunc, _dtype=np.float32):
    self.ins = ins
    self.outs = outs
    self.ws = [ [ Weight(next(weight_gen)) for j in range(outs) ] for i in range(ins) ]
    self.bs = [ Bias(next(bias_gen)) for j in range(outs) ]
    self.activation = activation
    self.dtype = _dtype

  def forward(self, xs):
    hs = [0 for _ in range(len(self.bs))]

    for i in range(len(self.ws)):
      for j in range(len(self.ws[i])):
        #print((len(self.ws), len(self.ws[0])))
        hs[j] += self.ws[i][j].mul(xs[i])

    for j in range(len(self.bs)):
      hs[j] = self.bs[j].add(hs[j])

    #for j in range(len(self.activations)):
    #  hs[j] = self.activations[j](hs[j])

    hs = np.array(hs)

    self.bef_activation_hs = hs.copy()

    hs = self.activation(hs)

    return hs

  def backward(self, grads, lr, clear=True): # provides grad for each h
    # but first consider the layerfunc
    grads = self.activation.get_in_grads(self.bef_activation_hs.ravel(), grads) # in_grad to LayerFunc is an intermediate grad in Layer

   # print("CALCULATED GRADS:", grads)

    in_grads = [0 for _ in range(self.ins)]

    for i in range(len(self.ws)):
      for j in range(len(self.ws[i])):
        self.ws[i][j].grad = grads[j]
        self.ws[i][j].finalize_grad()
        in_grads[i] += self.ws[i][j].w * self.ws[i][j].grad
        #print("GRADS J", type(grads[j]))
        self.ws[i][j].update_w(lr)
        if clear: self.ws[i][j].clear_grad()

    for j in range(len(self.bs)):
      self.bs[j].grad = grads[j]
      self.bs[j].update_b(lr)
      if clear: self.bs[j].clear_grad()

    return in_grads

  def backward_adam(self, grads, lr, beta1, beta2, clear=True):
    # but first consider the layerfunc
    grads = self.activation.get_in_grads(self.bef_activation_hs, grads) # in_grad to LayerFunc is an intermediate grad in Layer

    in_grads = [0 for _ in range(self.ins)]

    for i in range(len(self.ws)):
      for j in range(len(self.ws[i])):
        self.ws[i][j].grad = grads[j]
        self.ws[i][j].finalize_grad()
        in_grads[i] += self.ws[i][j].w * self.ws[i][j].grad
        self.ws[i][j].adam_update_w(lr, beta1, beta2)
        if clear: self.ws[i][j].clear_grad()

    for j in range(len(self.bs)):
      self.bs[j].grad = grads[j]
      self.bs[j].adam_update_b(lr, beta1, beta2)
      if clear: self.bs[j].clear_grad()

    return in_grads








In [4]:
def normal_generator(seed=10, mu=0, sigma=1):
  rng = np.random.default_rng(seed)
  while True:
    yield rng.normal(mu, sigma)


def sigmoid_activation(x):
  return 1 / (1 + np.exp(-x))

def get_sigmoid_layerfunc(num, _dtype=np.float32):
  return LayerFunc(num, num, sigmoid_activation, _dtype)

In [5]:
# DATA GENERATION

# 3 inputs, 4 hiddens, 2 outputs
dim_in = 3
dim_hidden = 4
dim_out = 2
# x,y,z point classification task

line0init = np.array([0, 1, 0])
line0vec = np.array([1, 3, -5])

line1init = np.array([-1, 0, 0])
line1vec = np.array([3, -1, 2])

# closest distance between line 0 and line 1: https://www.geeksforgeeks.org/shortest-distance-between-two-lines-in-3d-space-class-12-maths/
cross01 = np.cross(line0vec, line1vec)
mulc = cross01.copy() * (line1init - line0init)
dist = np.sqrt(mulc.dot(mulc)) / np.sqrt(cross01.dot(cross01))
print("DIST:", dist)

def get_line(num:int):
  if num == 0:
    return line0init + (np.random.random()-0.5)*5*line0vec + np.random.normal(scale=0.2, size=(3, )).clip(-0.45, 0.45)*dist
  else:
    return line1init + (np.random.random()-0.5)*5*line1vec + np.random.normal(scale=0.2, size=(3, )).clip(-0.45, 0.45)*dist

'''
# we won't need these
def get_batch(batch_size=1000):
  labels = np.random.randint(0, 2, size=(batch_size,1))
  xss = []
  for label in labels:
    xss.append(get_line(label).reshape((-1,1)))
  xs = np.stack(xss)
  ys = np.stack([labels==0, labels==1], axis=1)
  #print(labels[:5])
  #print(ys[:5])
  return xs, ys.astype(np.int32)


# test generation
xs, ys = get_batch(1000)
print(xs.shape)
print(ys.shape)
'''

def xy_gen(seed=10):
  while True:
    res = random.getrandbits(1)
    x = get_line(res)#.tolist()
    y = [int(res==0), int(res==1)]
    yield x, y

xyg = xy_gen()

DIST: 0.8623164985025764


In [6]:
x, y = next(xyg)
x

array([  2.14200636,   7.92984055, -11.77581845])

In [7]:
# initialize params

l1 = Layer(dim_in, dim_hidden, normal_generator(), normal_generator(), get_sigmoid_layerfunc(dim_hidden))

def softmax(p):
  t = np.exp(p[0]) + np.exp(p[1])
  return np.stack([np.exp(p[0])/t, np.exp(p[1])/t])

l2 = Layer(dim_hidden, dim_out, normal_generator(), normal_generator(), LayerFunc(dim_out, dim_out, softmax))


def get_lr(epochNum:int=0):
  return (5e-4)*(np.exp(-0.00004*epochNum))

In [8]:
# RUN

# prep eval set
eval_num = 1000
evals = [next(xyg) for _ in range(eval_num)]

def evaluate(epochNum:int = None):

  losses = []
  correct = 0
  for x, y in evals:
    ypred = l2.forward(l1.forward(x))

    # get loss
    losses.append(CEloss(ypred[0], ypred[1], y[0], y[1]))

    # accuracy
    if ypred[0] > ypred[1]:
      # predicts 0
      correct += y[0]
    else:
      # predicts 1
      correct += y[1]

  #print("YPRED", ypred, "Y", y, "LOSS", CEloss(ypred[0], ypred[1], y[0], y[1])) # show one sample

  loss = sum(losses)/len(losses)


  # display
  if epochNum is None:
    print('EVAL LOSS:', loss)
  else:
    print('EPOCH', epochNum, 'AVG EVAL LOSS:', loss)

  print("ACCURACY:", correct/len(losses))

  return loss




train_epoch_losses = []
eval_epoch_losses = []

epoch = 0
runs_per_epoch = 10000
avgepochloss = 10
while avgepochloss > 0.02:

  epochtotalloss = 0

  for i in range(runs_per_epoch):
    # get data
    x, y = next(xyg)
    #print(x)
    ypred = l2.forward(l1.forward(x))

    # get loss
    loss = CEloss(ypred[0], ypred[1], y[0], y[1])
    #print("RAW LOSS:", CEloss(p_s[:,0,0], p_s[:,1,0], y[:,0,0], y[:,1,0]))
    #print("LOSS:", loss)

    #print(ypred[0], ypred[1], y[0], y[1], loss)

    #1/0

    # update params
    lr = get_lr(epoch)
    grads = l2.backward(calc_grads_intinputs(CEloss, ypred.ravel().tolist(), y, fn_out=loss), lr)
    l1.backward(grads, lr)


    epochtotalloss += loss

  avgepochloss = epochtotalloss/runs_per_epoch

  if epoch%5 == 0:
    eval_loss = evaluate(epoch)
    print(epoch, "     AVG TRAIN LOSS:", avgepochloss)
    eval_epoch_losses.append(eval_loss) # includes epoch 0
  elif epoch < 5:
    print("EPOCH", epoch, "AVG TRAIN LOSS:", avgepochloss)

  epoch += 1
  train_epoch_losses.append(avgepochloss)


EPOCH 0 AVG EVAL LOSS: 0.9324091275991319
ACCURACY: 0.523
0      AVG TRAIN LOSS: 0.9699833474755578
EPOCH 1 AVG TRAIN LOSS: 0.8999815883259198
EPOCH 2 AVG TRAIN LOSS: 0.843183332795122
EPOCH 3 AVG TRAIN LOSS: 0.7783507526883707
EPOCH 4 AVG TRAIN LOSS: 0.7050119367175717
EPOCH 5 AVG EVAL LOSS: 0.625263763531739
ACCURACY: 0.773
5      AVG TRAIN LOSS: 0.6360996234843231
EPOCH 10 AVG EVAL LOSS: 0.3570850863128287
ACCURACY: 0.989
10      AVG TRAIN LOSS: 0.36397199841555555
EPOCH 15 AVG EVAL LOSS: 0.22653924547680881
ACCURACY: 0.992
15      AVG TRAIN LOSS: 0.23150470956521116
EPOCH 20 AVG EVAL LOSS: 0.1581691664303159
ACCURACY: 0.995
20      AVG TRAIN LOSS: 0.15911437987640062
EPOCH 25 AVG EVAL LOSS: 0.11869035663330425
ACCURACY: 0.996
25      AVG TRAIN LOSS: 0.11684426508755164
EPOCH 30 AVG EVAL LOSS: 0.09429298814877808
ACCURACY: 0.999
30      AVG TRAIN LOSS: 0.0907169060156591
EPOCH 35 AVG EVAL LOSS: 0.07696269535389677
ACCURACY: 0.999
35      AVG TRAIN LOSS: 0.07618056947605732
EPOCH 40 

KeyboardInterrupt: 

I think it's pretty clear that it's learning and functional, so i just terminated it 
also 100% accuracy yay 

do note that since i keep generating more train data, this is theoretically able to converge since the train data will cover the entire distribution. 
and it should be nearly linearly separable - so perceptron model should be enough for this task 
